In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [414]:
import data_util.preprocess 
importlib.reload(data_util.preprocess)
from data_util.preprocess import *



import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config

importlib.reload(data_util.batcher)
importlib.reload(model)
importlib.reload(train)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)

from data_util.batcher import *
from evaluate import *
from model import *
from train import *
from train_util import *
from data_util.data import *



In [3]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

In [4]:
!pwd

/Users/rowancassius/Desktop/capstone/LSTM_Summarizer


In [404]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data_fresh.tsv'
dat = pd.read_csv(data_path, sep='\t')

# fill nas
dat.fillna('', inplace=True)

# train/test split
np.random.seed(111)
dat = dat.sample(frac=1)
train_size = int(.8*dat.shape[0])
train_data = dat[:train_size]
test_data = dat[train_size:]


In [405]:
pd.set_option('display.max_colwidth', -1)

In [406]:
train_data.sample(10)

,#,Task,Context,TaskSentence,Summary,Labeler,NoRequestInContext,Urgent,NotRequest,Unsure/Discuss,RandomNumber
351,352,Call me a NX1 68 offer for the term and volumes listed.,Tom -,Call me a NX1 68 offer for the term and volumes listed.,Cal SENDER,Rowan,,0,,,0.671638
1265,5369,Jeff/Eric - Please take a look at the e-mail below and help out on the,"Section 6.7.2 Invoices should be addressed to:. Enron North America 1400 Smith St Houston, TX 77002 ATTN: Theresa Vos 713-3458-173. Attached in the latest draft of the GE-ENA breakout contract.. The lawyers need a few more items to complete before execution.. Please send to Kay Mann and copy me on responses.",Jeff/Eric - Please take a look at the e-mail below and help out on the following... accurate description of facitliy-pg 1 questions regarding the exhibits Address for Section 25.1 and 25.3,Review draft of GE-ENA breakout contract and help with accurate description of facility,Natalie,1,0,0,,0.172902
1354,5458,"I had to take Chelsey to sports, can you give me call and update me on the meeting.","Lynn, Steve January mentioned that I missed a meeting last night about Oneok concerning Bushton.","I had to take Chelsey to sports, can you give me call and update me on the meeting.",Call SENDER about Oneok concerning Bushton,Natalie,1,0,0,,0.001874
514,1706,Please forward: Room Change,"Kim, please forward to the traders.. I think the schedulers have this already.",Please forward: Room Change,Forward Room Change,Percy,,,,,0.099476
970,5073,Please sign me up.,"Shawn,",Please sign me up.,Sign SENDER up,Natalie,1,0,0,,0.442543
1092,5196,Please give me a call this week I need some info.,,Please give me a call this week I need some info.,Call SENDER,Natalie,1,0,0,,0.525904
889,3656,ortant to us -- please read it.,"In this exciting sequel, Fu teams up with female mixing phenom DJ Zee for a four-city Ninja DJ Tour a cross the US.. Play now!. Brought to you by Ford Focus.. 09 IMAGE 09 09 09 09 09 ATTENTION MEMBERS IMAGE When the redesigned Shockwave.com launched on August 21, it came with an updated privacy policy.",Your privacy is imp ortant to us -- please read it.,Read privacy policy.,George,1,,,,0.868424
882,3649,Go ahead and order the ac for #27.,,Go ahead and order the ac for #27.,Order ac for #27.,George,,,,,0.477414
526,1718,Will you bring your jersey so your sub can wear it?,,Will you bring your jersey so your sub can wear it?,Bring jersey,Percy,1,,,,0.416095
338,339,TK could you update these prices for me for the Stan Presentation one more time - it's due to Tracy on Thurs ?,,TK could you update these prices for me for the Stan Presentation one more time - it's due to Tracy on Thurs ?,Update these prices,Rowan,,0,,,0.389516


##  Vocabulary Curation

In [407]:
vocab_sents = [vocab_process_text(s) for s in 
               train_data.Context.values + train_data.TaskSentence.values]

In [411]:
# fit tokenizer
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(vocab_sents + list(ENT_TAGS))
assert all([t in tokenizer.word_index for t in ENT_TAGS])

In [412]:
!pwd

/Users/rowancassius/Desktop/capstone/LSTM_Summarizer


In [415]:
words2vocabfile(tokenizer.word_index.keys(), os.path.join(config.log_root, 'data/vocab.txt'))

In [416]:
vocab = Vocab.from_vocab_file(os.path.join(config.log_root, 'data/vocab.txt'))

Finished constructing vocabulary of 4654 total words. Last word added: a


In [393]:
i = 925
context = train_data.loc[i]['Context']
task    = train_data.loc[i]['TaskSentence']
summary = train_data.loc[i]['Summary']


context = article_process_text(context)
task = article_process_text(task)
summary = summary_process_text(summary)

In [394]:
import data_util.batcher
importlib.reload(data_util.batcher)
from data_util.batcher import *

In [395]:
ex = TaskExample(context, task, summary, vocab)

In [396]:
ex.pretty_print()

CONTEXT: Message sent from the pjm-customer-info mailing list at pjm-customer-info@majordomo.pjm.com:. The new PJM Enhanced Energy Scheduler EES will go into production at 10:00 this morning 4/17/00 at https://ees.pjm.com/mui/index.htm . Schedules may be submitted for energy that begins on or after tomorrow, 4/18/00. 

TASK:    For questions please contact PJM at 610-6662-270 . 

SUMMARY: contact PJM at 610-6662-270 with questions


In [397]:
[vocab.id2word(i) for i in ex.dec_input]

['[START]', 'contact', 'ORG', 'at', 'PHONENUMBER', 'with', 'questions']

In [398]:
len(ex.enc_input)

51

In [402]:
ex.dec_input[1:]

[230, 970, 2988, 1387, 2268, 3035]

In [401]:
ex.target[:-1]

[230, 4178, 2988, 4184, 2268, 3035]

In [403]:
ex.entity_label_map

{'pjm-customer-info@majordomo.pjm.com': 'EMAILADDRESS',
 'PJM Enhanced Energy Scheduler EES': 'ORG',
 '10:00 this morning': 'TIME',
 'https://ees.pjm.com/mui/index.htm': 'WEBSITE',
 'PJM': 'ORG',
 '610-6662-270': 'PHONENUMBER'}

In [392]:
ex.article_oovs

['Benji', ' ', 'Dinner', '22nd']

In [357]:
doc = nlp("Hey George, please email me asap at rowan@gmail.net. Take the survey at extraction.com. Also my friend at Google wants you to edit Article III, and call him at (445) 7680099")

In [345]:
{e.text: e.label_ for e in doc.ents}

{'George': 'PERSON',
 'rowan@gmail.net': 'EMAILADDRESS',
 'extraction.com': 'WEBSITE',
 'Google': 'ORG',
 'Article III': 'PRODUCT',
 '(445) 7680099': 'PHONENUMBER'}

In [346]:
list(doc)

[Hey,
 George,
 ,,
 please,
 email,
 me,
 asap,
 at,
 rowan@gmail.net,
 .,
 Take,
 the,
 survey,
 at,
 extraction.com,
 .,
 Also,
 my,
 friend,
 at,
 Google,
 wants,
 you,
 to,
 edit,
 Article III,
 ,,
 and,
 call,
 him,
 at,
 (445) 7680099]

In [221]:
nlp('Term Sheet').ents

()

In [ ]:
from spacy.tokenizer import wor